In [18]:
#subproblem 3
import pandas as pd

# Load datasets
pit_stops = pd.read_csv("pit_stops.csv")
results = pd.read_csv("results.csv")
races = pd.read_csv("races.csv")
drivers = pd.read_csv("drivers.csv")
constructors = pd.read_csv("constructors.csv")

# Merge datasets to get race results with pit stop data
merged_data = pit_stops.merge(results, on=['raceId', 'driverId'], how='left')
merged_data = merged_data.merge(races, on='raceId', how='left')
merged_data = merged_data.merge(drivers, on='driverId', how='left')
merged_data = merged_data.merge(constructors, on='constructorId', how='left')

# Instead of selecting columns, rename the relevant columns
# to match the desired names for analysis
merged_data = merged_data.rename(columns={
    'name_y': 'name',  # Assuming 'name_y' is the constructor name
    'milliseconds_x': 'milliseconds'  # Assuming 'milliseconds_x' is from pit_stops
})

# Now you can select the columns as intended
merged_data = merged_data[['raceId', 'year', 'round', 'circuitId', 'driverId', 'surname','constructorId', 'name', 'positionOrder', 'lap', 'milliseconds']]

In [19]:
#check how many pit stops correlate with top 3 finishes vs. mid-field vs. bottom finishes.
# Count the number of pit stops per driver in each race
pit_stop_counts = merged_data.groupby(['raceId', 'driverId'])['lap'].count().reset_index()
pit_stop_counts.rename(columns={'lap': 'total_pit_stops'}, inplace=True)

# Merge with finishing positions
pit_stop_analysis = pit_stop_counts.merge(results[['raceId', 'driverId', 'positionOrder']],
                                          on=['raceId', 'driverId'], how='left')

# Average pit stops for different finishing groups
optimal_pit_stops = pit_stop_analysis.groupby('positionOrder')['total_pit_stops'].mean().reset_index()
pit_stop_counts['total_pit_stops'] = pit_stop_counts['total_pit_stops'].astype(int)
print(optimal_pit_stops)


    positionOrder  total_pit_stops
0               1         1.936842
1               2         1.940351
2               3         1.996491
3               4         2.042105
4               5         2.010526
5               6         1.971930
6               7         1.975439
7               8         1.940351
8               9         2.007018
9              10         2.007042
10             11         2.042105
11             12         2.084211
12             13         2.154386
13             14         2.176056
14             15         2.212014
15             16         2.194245
16             17         2.162963
17             18         2.091286
18             19         2.053659
19             20         1.767742
20             21         1.927536
21             22         2.040816
22             23         1.782609
23             24         1.500000


In [20]:
#if early, mid, or late-race pit stops affect race outcomes.
import numpy as np
# Categorize pit stops into Early (1st 30% laps), Mid (30%-70%), Late (Last 30%)
merged_data['race_phase'] = np.where(merged_data['lap'] <= merged_data['lap'].max() * 0.3, 'Early',
                                     np.where(merged_data['lap'] <= merged_data['lap'].max() * 0.7, 'Mid', 'Late'))

# Average finishing positions for each pit stop timing strategy
pit_stop_timing_analysis = merged_data.groupby('race_phase')['positionOrder'].mean().reset_index()
print(pit_stop_timing_analysis)


  race_phase  positionOrder
0      Early      10.763084
1       Late       9.847561
2        Mid      10.285337


In [21]:
#which teams perform the fastest stops
# Get average pit stop time per team
team_pit_stop_efficiency = merged_data.groupby('name')['milliseconds'].mean().reset_index()
team_pit_stop_efficiency = team_pit_stop_efficiency.sort_values(by='milliseconds')

print(team_pit_stop_efficiency.head(10))  # Fastest teams


           name  milliseconds
21       Virgin  24236.194805
9         Lotus  24444.211765
10     Lotus F1  32463.750877
7           HRT  32677.766667
4      Caterham  33924.618257
12     Marussia  34396.836910
20   Toro Rosso  44486.983895
6   Force India  50845.770115
19       Sauber  57248.313559
18      Renault  63261.833747


In [22]:
#whether fast pit stops improve finishing positions.
# Merge pit stop data with results
merged_data['total_pit_time'] = merged_data.groupby(['raceId', 'driverId'])['milliseconds'].transform('sum')

# Average pit stop time per finishing position
pit_stop_impact = merged_data.groupby('positionOrder')['total_pit_time'].mean().reset_index()

print(pit_stop_impact)


    positionOrder  total_pit_time
0               1   270354.079710
1               2   268391.956600
2               3   275959.753954
3               4   255518.262887
4               5   283003.642234
5               6   269636.674377
6               7   290056.394316
7               8   291178.566004
8               9   292165.466783
9              10   295010.373684
10             11   310639.190722
11             12   274444.639731
12             13   271572.366450
13             14   266147.377023
14             15   246142.450479
15             16   258378.580328
16             17   231116.962329
17             18   147127.962302
18             19   120115.859857
19             20    87780.251825
20             21    96614.285714
21             22    95659.650000
22             23    54781.951220
23             24    42561.476190


In [ ]:
'''
Interpretations
Optimal pit stop count – Likely 1-2 stops for podium finishes, 3+ for midfield/bottom.
Timing impact – Early stops may be bad, mid-race or late stops could be better.
Fastest pit crews – Top teams have 2.0-2.5s stops, others might lose 1-2s per stop.
Pit stop efficiency and finishing position – Slower pit stops can ruin race chances.'''